Install dependencies

In [60]:
! pip install pandas;
! pip install pgmpy;


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Import dependencies

In [61]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from pgmpy.inference import VariableElimination

Initialise Data

In [62]:
Quali = "Qualifikation"
S = "Schnitt"
BL = "Bundesland"
M = "Mathe"
P = "Physik"
D = "Deutsch"
ST = "Schultyp"
OLTM = "OLT-Mathe"
OLTD = "OLT-Deutsch"
SFT = "Studierfähigkeitstest"
A = "Alter"
Geschl = "Geschlecht"
JEDE = "Jahreseinkommen der Eltern"
SB = "Staatsbürgerschaft"
SG = "Studiengang"
Absch= "Abschluss"

Import data from csv File

In [63]:
init_data = pd.read_csv('p001_1.csv', sep=';',dtype={
    Quali: 'string'
})
display(init_data)

,Qualifikation,Schnitt,Bundesland,Mathe,Physik,Deutsch,Schultyp,OLT-Mathe,OLT-Deutsch,Studierfähigkeitstest,Alter,Geschlecht,Jahreseinkommen der Eltern,Staatsbürgerschaft,Studiengang,Abschluss
0,Abitur,"2,7",Baden-Württemberg,"2,3","2,2","2,1",Allgemeinbildendes Gymnasium,63,62,685,19,m,47000,deutsch,Maschinenbau,3
1,Meister,"1,6",Nordrhein-Westfalen,keine,keine,keine,n.a.,36,59,n.a.,25,m,87000,deutsch,Maschinenbau,abgebrochen
2,Abitur,"1,8",Baden-Württemberg,1,"1,1","1,2",Wirtschaftsgymnasium,96,94,n.a.,18,w,115000,deutsch,Soziale Arbeit,"1,1"
3,Abitur,"1,1",Baden-Württemberg,"2,5","2,7","1,9",Technisches Gymnasium,70,76,n.a.,17,m,115000,deutsch,Elektrotechnik,"2,6"
4,Abitur,"1,4",Bayern,2,"2,1","1,4",Wirtschaftsgymnasium,65,82,n.a.,17,m,90000,deutsch,Elektrotechnik,"2,3"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Abitur,"1,4",Baden-Württemberg,"2,4","2,5","2,5",Allgemeinbildendes Gymnasium,60,65,n.a.,19,w,107000,deutsch,Elektrotechnik,"3,1"
96,Abitur,"2,6",Baden-Württemberg,"1,1","1,1","1,2",Technisches Gymnasium,100,94,n.a.,20,m,80000,deutsch,Wirtschaftswissenschaften,"1,5"
97,Abitur,"1,6",Baden-Württemberg,2,"2,2","1,7",Technisches Gymnasium,76,86,n.a.,19,m,208000,deutsch,Maschinenbau,"2,4"
98,Abitur,"2,6",Bayern,"1,5","1,4","1,1",Allgemeinbildendes Gymnasium,80,86,n.a.,18,m,110000,deutsch,Maschinenbau,"1,8"


Specify data types and handle missing values

In [64]:
# Define data types for numerical columns
numerical_columns = [S, M, D, P, OLTM, OLTD, SFT, A, JEDE]

for column in numerical_columns:
    init_data.loc[:, column] = pd.to_numeric(
        init_data[column], errors="coerce"
    )

# Handle missing values in categorical columns
categorical_columns = [Quali, BL, ST, Geschl, SB, SG]
for column in categorical_columns:
    # Replace missing values with the most frequent category
    init_data.loc[:, column].fillna(init_data[column].mode()[0], inplace=True)

C:\Users\robin\AppData\Local\Temp\ipykernel_41268\2205911213.py:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  init_data.loc[:, column] = pd.to_numeric(


Create Bayesian Network with Nodes and Edges and init_data

In [65]:
# Create Bayesian Network
G = BayesianNetwork()
G.add_nodes_from(init_data.columns)

# Add edges to the network
G.add_edges_from([
    (ST, SG),  # Schultyp → Studiengang
    (Quali, S),  # Qualifikation → Schnitt
    (S, M),  # Schnitt → Mathe
    (S, D),  # Schnitt → Deutsch
    (S, P),  # Schnitt → Physik
    (P, OLTM),  # Physik → OLT-Mathe
    (M, OLTM),  # Mathe → OLT-Mathe
    (D, OLTD),  # Deutsch → OLT-Deutsch
    (OLTM, Absch),  # OLT-Mathe → Abschluss
    (OLTD, Absch),  # OLT-Deutsch → Abschluss
    (BL, JEDE),  # Bundesland → Jahreseinkommen der Eltern
    (JEDE, SFT),  # Jahreseinkommen der Eltern → Studierfähigkeitstest
    (JEDE, SG),  # Jahreseinkommen der Eltern → Studiengang
    (SG, Absch),  # Studiengang → Abschluss
    (A, SFT),  # Alter → Studierfähigkeitstest
    (Geschl, SFT),  # Geschlecht → Studierfähigkeitstest
    (SB, SFT)  # Staatsbürgerschaft → Studierfähigkeitstest
])

# Use BayesianEstimator to learn the CPDs
G.fit(data=init_data, estimator=BayesianEstimator)

Make predictions with given input data

In [70]:
# Perform inference with incomplete input
# Example: Predict Abschluss for a student with Qualifikation = "Abitur",
# Schultyp = "Allgemeinbildendes Gymnasium", BL = "Baden-Württemberg",
# and A = 22
evidence = {
    Quali: "Abitur",
    ST: "Allgemeinbildendes Gymnasium",
    BL: "Bayern",
    A: 25
}

# Use Variable Elimination to make predictions
infer = VariableElimination(G)
q = infer.query(variables=[Absch], evidence=evidence)
print(q)

+------------------------+------------------+
| Abschluss              |   phi(Abschluss) |
+========================+==================+
| Abschluss(1,1)         |           0.0345 |
+------------------------+------------------+
| Abschluss(1,2)         |           0.0345 |
+------------------------+------------------+
| Abschluss(1,5)         |           0.0345 |
+------------------------+------------------+
| Abschluss(1,6)         |           0.0345 |
+------------------------+------------------+
| Abschluss(1,7)         |           0.0345 |
+------------------------+------------------+
| Abschluss(1,8)         |           0.0345 |
+------------------------+------------------+
| Abschluss(1,9)         |           0.0345 |
+------------------------+------------------+
| Abschluss(2)           |           0.0345 |
+------------------------+------------------+
| Abschluss(2,1)         |           0.0345 |
+------------------------+------------------+
| Abschluss(2,2)         |        